In [1]:
!pip install faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from datetime import date, datetime
import uuid
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text
import random
from faker import Faker


## Restaura arquivo banco de dados

In [ ]:
db_path = '/content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db'

# Verifica se o arquivo existe
if os.path.exists(db_path):
    # Remove o arquivo existente
    os.remove(db_path)
    print(f"Arquivo {db_path} removido.")
else:
    print(f"Arquivo {db_path} não encontrado.")

# Cria um novo arquivo SQLite
with open(db_path, 'w') as file:
    pass  # Não adiciona nada ao arquivo

print(f"Arquivo {db_path} criado ou substituído.")

Arquivo /content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db removido.
Arquivo /content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db criado ou substituído.


## Abre a conexão com o banco de dados

In [ ]:
# Conexão ao banco de dados SQLite para criação do schema e dados fake
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

## Define o Schema dobanco de dados

In [ ]:
# Script para criar as tabelas
schema_script = """
CREATE TABLE IF NOT EXISTS CAD_PESSOA (
    ID_PESSOA TEXT UNIQUE PRIMARY KEY,
    NOME VARCHAR(100) NOT NULL,
    CPF VARCHAR(11) UNIQUE NOT NULL,
    DATA_NASC DATE NOT NULL,
    GENERO VARCHAR(20),
    ORIENTACAO_SEXUAL VARCHAR(20),
    RACA_ETNIA VARCHAR(50),
    ESCOLARIDADE VARCHAR(50),
    ESTADO_CIVIL VARCHAR(20),
    FAIXA_DE_RENDA VARCHAR(50),
    AREA_DE_ATUACAO VARCHAR(100),
    TELEFONE VARCHAR(15),
    EMAIL VARCHAR(100) UNIQUE,
    URL_REDE_SOCIAL VARCHAR(255),
    ENDERECO VARCHAR(255),
    CEP VARCHAR(10),
    LOGRADOURO VARCHAR(100),
    NUMERO VARCHAR(10),
    BAIRRO VARCHAR(100),
    CIDADE VARCHAR(100),
    ESTADO VARCHAR(2),
    PAIS VARCHAR(100),
    DATA_VISITA DATE
);

CREATE TABLE IF NOT EXISTS ONU_ODS (
    ID_ODS INTEGER PRIMARY KEY AUTOINCREMENT,
    NOME_ODS VARCHAR(100) NOT NULL
);

CREATE TABLE IF NOT EXISTS DOMINIOS (
    ID_DOMINIO INTEGER PRIMARY KEY AUTOINCREMENT,
    NOME_DOMINIO VARCHAR(100) NOT NULL,
    RESPONSAVEL_DOMINIO VARCHAR(100) NOT NULL
);

CREATE TABLE IF NOT EXISTS SERVICOS (
    ID_SERVICO INTEGER PRIMARY KEY AUTOINCREMENT,
    TITULO_SERVICO VARCHAR(100) NOT NULL
);

CREATE TABLE IF NOT EXISTS PROGRAMAS (
    ID_PROGRAMA TEXT UNIQUE PRIMARY KEY,
    ID_DOMINIO INTEGER NOT NULL,
    TITULO_PROGRAMA VARCHAR(100) NOT NULL,
    FOREIGN KEY (ID_DOMINIO) REFERENCES DOMINIOS(ID_DOMINIO)
);

CREATE TABLE IF NOT EXISTS PROGRAMA_ODS (
    ID_PROGRAMA TEXT NOT NULL,
    ID_ODS INTEGER NOT NULL,
    PRIMARY KEY (ID_PROGRAMA, ID_ODS),
    FOREIGN KEY (ID_PROGRAMA) REFERENCES PROGRAMAS(ID_PROGRAMA),
    FOREIGN KEY (ID_ODS) REFERENCES ONU_ODS(ID_ODS)
);

CREATE TABLE IF NOT EXISTS INSCRICOES (
    ID_INSCRICAO TEXT UNIQUE PRIMARY KEY,
    ID_PROGRAMA TEXT NOT NULL,
    ID_PESSOA TEXT NOT NULL,
    DATA_INSCRICAO DATE NOT NULL,
    FOREIGN KEY (ID_PROGRAMA) REFERENCES PROGRAMAS(ID_PROGRAMA),
    FOREIGN KEY (ID_PESSOA) REFERENCES CAD_PESSOA(ID_PESSOA)
);

CREATE TABLE IF NOT EXISTS PARTICIPANTE_PROGRAMA (
    ID_PROGRAMA TEXT NOT NULL,
    ID_INSCRICAO TEXT NOT NULL,
    PRIMARY KEY (ID_PROGRAMA, ID_INSCRICAO),
    FOREIGN KEY (ID_PROGRAMA) REFERENCES PROGRAMAS(ID_PROGRAMA),
    FOREIGN KEY (ID_INSCRICAO) REFERENCES INSCRICOES(ID_INSCRICAO)
);

CREATE TABLE IF NOT EXISTS AGENDAMENTO_SERVICO (
    ID_AGENDAMENTO INTEGER PRIMARY KEY AUTOINCREMENT,
    ID_PESSOA TEXT NOT NULL,
    ID_SERVICO INTEGER NOT NULL,
    DATETIME_AGENDAMENTO DATETIME NOT NULL,
    FOREIGN KEY (ID_PESSOA) REFERENCES CAD_PESSOA(ID_PESSOA),
    FOREIGN KEY (ID_SERVICO) REFERENCES SERVICOS(ID_SERVICO)
);
"""

# Executar o script SQL para criar as tabelas
cursor.executescript(schema_script)

# Confirmar as alterações e fechar a conexão
conn.commit()

print("Tabelas criadas com sucesso!")

Tabelas criadas com sucesso!


In [ ]:
# Consultar as tabelas existentes
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Recuperar e imprimir os nomes das tabelas
tables = cursor.fetchall()
print("Tabelas no banco de dados:")
for table in tables:
    print(table[0])


Tabelas no banco de dados:
CAD_PESSOA
ONU_ODS
sqlite_sequence
DOMINIOS
SERVICOS
PROGRAMAS
PROGRAMA_ODS
INSCRICOES
PARTICIPANTE_PROGRAMA
AGENDAMENTO_SERVICO


## CAD_PESSOA

In [ ]:
fake = Faker('pt_BR')

# Função para gerar dados fictícios e inseri-los na tabela CAD_PESSOA
def insert_into_cad_pessoa(qtd_registros=10):
    bairros = [
          "Agamenon Magalhães",
          "Alto do Moura",
          "Boa Vista",
          "Caiucá",
          "Cedro",
          "Centro",
          "Cidade Jardim",
          "Cohab I",
          "Cohab II",
          "Divinópolis",
          "Indianópolis",
          "Jardim Panorama",
          "João Mota",
          "Lagoa do Algodão",
          "Maurício de Nassau",
          "Monte Bom Jesus",
          "Nova Caruaru",
          "Petrópolis",
          "Pinheirópolis",
          "Salgado",
          "Rendeiras",
          "Santa Clara",
          "São Francisco",
          "São José",
          "São João da Escócia",
          "Universitário",
          "Vassoural",
          "Vila Kennedy",
          "Xique xique",
        ]
    emails_gerados = set()  # Conjunto para garantir unicidade dos e-mails

    for _ in range(qtd_registros):
        id_pessoa = str(uuid.uuid4())
        nome = fake.name()
        cpf = fake.cpf()
        data_nasc = fake.date_of_birth(minimum_age=18, maximum_age=90).strftime('%Y-%m-%d')
        genero = random.choice(['Masculino', 'Feminino', 'Outro'])
        orientacao_sexual = random.choice(['Heterossexual', 'Homossexual', 'Bissexual', 'Outro'])
        raca_etnia = random.choice(['Branca', 'Preta', 'Parda', 'Amarela', 'Indígena'])
        escolaridade = random.choice(['Ensino Fundamental', 'Ensino Médio', 'Ensino Superior', 'Pós-graduação'])
        estado_civil = random.choice(['Solteiro', 'Casado', 'Divorciado', 'Viúvo'])
        faixa_de_renda = random.choice(['Até 1 salário mínimo', '1 a 3 salários mínimos', '3 a 5 salários mínimos', 'Mais de 5 salários mínimos'])
        area_de_atuacao = random.choice(['Tecnologia', 'Educação', 'Saúde', 'Comércio', 'Indústria', 'Serviços'])
        telefone = fake.phone_number()
        email = fake.email()

        # Garantir que o e-mail seja único
        while email in emails_gerados:
            email = fake.email()
        emails_gerados.add(email)

        url_rede_social = fake.url()
        endereco = fake.street_address()
        cep = fake.postcode()
        logradouro = fake.street_name()
        numero = str(fake.building_number())
        bairro = random.choice(bairros)
        cidade = 'Caruaru'
        estado = 'PE'
        pais = 'Brasil'
        data_visita = fake.date_this_year().strftime('%Y-%m-%d')

        # Inserir os dados na tabela CAD_PESSOA
        cursor.execute('''
            INSERT INTO CAD_PESSOA (
                ID_PESSOA, NOME, CPF, DATA_NASC, GENERO, ORIENTACAO_SEXUAL, RACA_ETNIA, ESCOLARIDADE, ESTADO_CIVIL,
                FAIXA_DE_RENDA, AREA_DE_ATUACAO, TELEFONE, EMAIL, URL_REDE_SOCIAL, ENDERECO, CEP, LOGRADOURO,
                NUMERO, BAIRRO, CIDADE, ESTADO, PAIS, DATA_VISITA
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (id_pessoa, nome, cpf, data_nasc, genero, orientacao_sexual, raca_etnia, escolaridade, estado_civil, faixa_de_renda,
              area_de_atuacao, telefone, email, url_rede_social, endereco, cep, logradouro, numero, bairro, cidade, estado,
              pais, data_visita))

    # Confirmar as alterações
    conn.commit()

# Inserir 1000 registros fictícios na tabela CAD_PESSOA
insert_into_cad_pessoa(1000)


In [ ]:
%load_ext sql
%sql sqlite:///{db_path}

In [ ]:
%sql SELECT COUNT(*) AS CONTAGEM_PESSOAS FROM CAD_PESSOA


 * sqlite:////content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db
Done.


CONTAGEM_PESSOAS
1000


## ONU_ODS

In [ ]:
ods = [
    "Erradicação da pobreza",
    "Fome zero e agricultura sustentável",
    "Saúde e bem-estar",
    "Educação de qualidade",
    "Igualdade de gênero",
    "Água potável e saneamento",
    "Energia limpa e sustentável",
    "Trabalho decente e inclusão social",
    "Indústria, inovação e infraestrutura",
    "Redução das desigualdades",
    "Cidades e comunidades sustentáveis",
    "Consumo e produção sustentáveis",
    "Ação contra mudança global do clima",
    "Vida na água",
    "Vida terrestre",
    "Paz, justiça e participação humanitária",
    "Parcerias e meios de implementação",
]

def insert_into_onu_ods(ods_list):

    for ods in ods_list:
        cursor.execute('''
            INSERT INTO ONU_ODS (NOME_ODS)
            VALUES (?)
        ''', (ods,))

    conn.commit()

insert_into_onu_ods(ods)


## DOMINIOS

In [ ]:
# Dicionário que mapeia cada domínio ao seu respectivo responsável
mapa_dominio_responsavel = {
    "Gestão": "Pâmela",
    "Inovação Aberta": "Lais",
    "Equidade de gênero": "Gabi",
    "Empreendedorismo": "Mirelli",
    "Suporte em TI": "Lucas",
    "Comunicação e Marketing": "Bono"
}

def inserir_into_dominios(map):


    nomes_dominios = list(map.keys())
    responsaveis_dominio = list(map.values())
    qtd_dominios = len(nomes_dominios)

    for i in range(qtd_dominios):

        # Selecionar o responsável correspondente ao domínio
        responsavel_dominio = responsaveis_dominio[i]
        nome_dominio = nomes_dominios[i]

        cursor.execute('''
            INSERT INTO DOMINIOS (NOME_DOMINIO, RESPONSAVEL_DOMINIO)
            VALUES (?, ?)
        ''', (nome_dominio, responsavel_dominio))

    conn.commit()


inserir_into_dominios(mapa_dominio_responsavel)

In [ ]:
%sql SELECT * FROM DOMINIOS

 * sqlite:////content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db
Done.


ID_DOMINIO,NOME_DOMINIO,RESPONSAVEL_DOMINIO
1,Gestão,Pâmela
2,Inovação Aberta,Lais
3,Equidade de gênero,Gabi
4,Empreendedorismo,Mirelli
5,Suporte em TI,Lucas
6,Comunicação e Marketing,Bono


## SERVICOS

In [ ]:
servicos = [
    "Coworking",
    "Laboratórios de Criação e Prototipagem",
    "Capacitação e Formação",
    "Apoio a Startups",
    "Eventos e Networking",
]

def insert_into_servicos(lista_servicos):

    for servico in lista_servicos:
        cursor.execute('''
            INSERT INTO SERVICOS (TITULO_SERVICO)
            VALUES (?)
        ''', (servico,))

    conn.commit()

insert_into_servicos(servicos)

## PROGRAMAS

In [ ]:
programas_dominios = {
    "Clubes de Programação": 3,
    "Elas que Lideram": 3,
    "Mulheres no Alto": 3,
    "Minas Data Lab": 3,
    "Transforma": 3,
    "Minas na Produção": 3,
    "Agreste Futurista": 3,
    "Observatório do amanhã":2,
    "Desafios de sustentabilidade": 2,
    "MMA (moda x música x audiovisual)": 2,
    ".GOV + Dotlab": 2,
    "Ciclo de inovação aberta": 2,
    "Incubação":4,
    "Mind the Bizz": 4,
    "Agreste Moda": 4,
}

def insert_into_programa():

    titulos_programas = list(programas_dominios.keys())
    dominios_programas = list(programas_dominios.values())
    qtd_programas = len(titulos_programas)

    for i in range(qtd_programas):

        id_programa = str(uuid.uuid4())
        titulo_programa = titulos_programas[i]
        id_dominio_programa = dominios_programas[i]
        cursor.execute('''
            INSERT INTO PROGRAMAS (ID_PROGRAMA, ID_DOMINIO, TITULO_PROGRAMA)
            VALUES (?, ?, ?)
        ''', (id_programa, id_dominio_programa, titulo_programa))

    conn.commit()

insert_into_programa()

In [ ]:
%sql SELECT * FROM PROGRAMAS ORDER BY ID_DOMINIO

 * sqlite:////content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db
Done.


ID_PROGRAMA,ID_DOMINIO,TITULO_PROGRAMA
1dd9f65c-2fb0-4912-ae25-b7f29c96498a,2,Observatório do amanhã
098ad163-82c0-4218-bfb5-bb5413668c09,2,Desafios de sustentabilidade
0ed1d47f-a10f-4760-a1e4-08d430f6bc16,2,MMA (moda x música x audiovisual)
6df60345-cd45-48ce-8818-afadf9015c36,2,.GOV + Dotlab
6d7256b4-ccc1-46d0-979c-98c03ce20e0a,2,Ciclo de inovação aberta
0c1081d6-b26e-478c-887e-d21e90ab48e6,2,Observatório do amanhã
12f2b61e-7eb3-487a-a126-230d12abfe48,2,Desafios de sustentabilidade
6805cba6-da8e-415d-ab52-941edb3aecd8,2,MMA (moda x música x audiovisual)
55a1e86f-f969-4700-a52b-acf976b84be1,2,.GOV + Dotlab
24a40858-5f6a-4459-ad5a-70729eb8319a,2,Ciclo de inovação aberta


## PROGRAMA_ODS

In [ ]:
## Todo

## INSCRICOES

In [ ]:
def insert_into_inscricoes_programas(qtd_registros=10):
    cursor.execute("SELECT ID_PESSOA FROM CAD_PESSOA")
    pessoas = cursor.fetchall()
    cursor.execute("SELECT ID_PROGRAMA FROM PROGRAMAS")
    programas = cursor.fetchall()

    # Define o intervalo de datas
    data_inicio = date(2024, 1, 15)
    data_fim = date(2024, 8, 1)

    for _ in range(qtd_registros):
        id_inscricao = str(uuid.uuid4())
        id_programa = random.choice(programas)[0]
        id_pessoa = random.choice(pessoas)[0]
        data_inscricao = fake.date_between(start_date=data_inicio, end_date=data_fim).strftime('%Y-%m-%d')

        cursor.execute('''
            INSERT INTO PARTICIPANTE_PROGRAMA (ID_INSCRICAO, ID_PROGRAMA, ID_PESSOA, DATA_INSCRICAO)
            VALUES (?, ?, ?, ?)
        ''', (id_inscricao, id_programa, id_pessoa, data_inscricao))

    conn.commit()

insert_into_inscricoes_programas(350)

In [ ]:
%sql SELECT * FROM INSCRICOES LIMIT 5

 * sqlite:////content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db
Done.


ID_INSCRICAO,ID_PROGRAMA,ID_PESSOA,DATA_INSCRICAO
c8479adc-03bf-4405-b5bb-f33cde50dbaf,ff937ac9-bc9a-4c36-86ab-06bdcc98cb38,1b9a9b3a-dc52-4977-a90a-234de1ae172f,2024-03-20
9114737d-c8ee-4cc5-b957-aeb4d0e03f18,2b103878-3ae4-4882-91c0-67d143a33982,4effb521-622d-4b2e-9f06-c822c1ed3822,2024-06-14
6821ace3-40cb-475e-8e85-9af326c9956e,55a1e86f-f969-4700-a52b-acf976b84be1,318bed70-bd5f-4d54-be64-426cc067a858,2024-07-24
859e8c73-98c7-44b5-a5ea-fe4776c1a564,e4792d2b-757d-4ec3-b4b0-7a728921c376,128b5852-beef-404c-8fac-df965db91988,2024-03-31
3c3ad30a-fc48-4738-aa29-e9e93cd386fa,098ad163-82c0-4218-bfb5-bb5413668c09,afdfe0c2-4186-4582-a877-0dbd64a4f00d,2024-06-06


In [ ]:
%%sql
SELECT P.nome
FROM CAD_PESSOA P
JOIN INSCRICOES I
ON P.ID_PESSOA = I.ID_PESSOA
-- WHERE I.ID_PESSOA = "1b9a9b3a-dc52-4977-a90a-234de1ae172f"
LIMIT 10


 * sqlite:////content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db
Done.


NOME
Dr. Yago Peixoto
Luiz Fernando Nunes
Maria Sophia Novais
Lucca Aparecida
Sra. Ana Júlia Montenegro
Lucca Sales
Dr. Lucca Lopes
Ana Laura Mendonça
Dra. Lívia Correia
Sr. Lorenzo Nascimento


## PARTICIPANTE_PROGRAMA

In [ ]:
def insert_into_programas_participante(qtd_registros=10):
    cursor.execute("SELECT ID_PROGRAMA FROM PROGRAMAS")
    programas = cursor.fetchall()
    cursor.execute("SELECT ID_INSCRICAO FROM INSCRICOES")
    inscricoes = cursor.fetchall()

    for _ in range(qtd_registros):
        id_inscricao = random.choice(inscricoes)[0]
        id_programa = random.choice(programas)[0]

        cursor.execute('''
            INSERT INTO INSCRICOES (ID_PROGRAMA, ID_INSCRICAO)
            VALUES (?, ?)
        ''', (id_inscricao, id_programa))

    conn.commit()

insert_into_inscricoes_programas(250)

In [ ]:
%sql SELECT * FROM PARTICIPANTE_PROGRAMA LIMIT 5

 * sqlite:////content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db
Done.


ID_PROGRAMA,ID_INSCRICAO


## AGENDAMENTO_SERVICO

In [ ]:
def insert_into_agendamento_servico(qtd_registros=10):
    cursor.execute("SELECT ID_SERVICO FROM SERVICOS")
    servicos = cursor.fetchall()
    cursor.execute("SELECT ID_PESSOA FROM CAD_PESSOA")
    pessoas = cursor.fetchall()

     # Define o intervalo de datas
    data_inicio = date(2024, 1, 15)
    data_fim = date(2024, 8, 1)

    for _ in range(qtd_registros):
        # id_agendamento = str(uuid.uuid4()) um dia talvez eu precise dessa variável
        id_servico = random.choice(servicos)[0]
        id_pessoa = random.choice(pessoas)[0]
        datetime_agendamento = fake.date_between(start_date=data_inicio, end_date=data_fim).strftime('%Y-%m-%d %H:%M:%S')

        cursor.execute('''
            INSERT INTO AGENDAMENTO_SERVICO (ID_SERVICO, ID_PESSOA, DATETIME_AGENDAMENTO)
            VALUES (?, ?, ?)
        ''', (id_servico, id_pessoa, datetime_agendamento))

    conn.commit()

insert_into_agendamento_servico(500)

## Consultando o banco de dodos com python

**Assim:**

In [ ]:
query = """
SELECT COUNT(*)
FROM CAD_PESSOA
"""
cursor.execute(query)
result = cursor.fetchall()

for row in result:
    print(row)


(1000,)


**Também assim:**

In [ ]:
query = """
SELECT *
FROM CAD_PESSOA
"""
cursor.execute(query)
result = cursor.fetchall()

# Instanciando um Objeto DataFrame:
df_cad_pessoa = pd.DataFrame(result, columns=[column[0] for column in cursor.description])
df_cad_pessoa.head()

,ID_PESSOA,NOME,CPF,DATA_NASC,GENERO,ORIENTACAO_SEXUAL,RACA_ETNIA,ESCOLARIDADE,ESTADO_CIVIL,FAIXA_DE_RENDA,...,URL_REDE_SOCIAL,ENDERECO,CEP,LOGRADOURO,NUMERO,BAIRRO,CIDADE,ESTADO,PAIS,DATA_VISITA
0,713970c2-540c-484d-b4c5-e432e5cce09c,Juan Teixeira,601.873.924-04,1958-06-20,Masculino,Heterossexual,Parda,Ensino Fundamental,Solteiro,Mais de 5 salários mínimos,...,https://rios.net/,"Núcleo da Paz, 96",27656106,Jardim Caldeira,139,Santa Clara,Caruaru,PE,Brasil,2024-03-19
1,25d88136-001f-4e37-8e2a-3de3f56bbf2c,Francisco Aragão,845.706.913-66,1950-07-25,Feminino,Bissexual,Preta,Ensino Superior,Casado,3 a 5 salários mínimos,...,http://vieira.br/,"Praia Gabriela Pastor, 89",46065-772,Residencial João Sales,70,São José,Caruaru,PE,Brasil,2024-03-10
2,83482d24-7cf1-4335-9f3a-c267e6b9f2c8,Otto Peixoto,867.051.243-26,1972-08-20,Outro,Outro,Preta,Ensino Fundamental,Casado,1 a 3 salários mínimos,...,http://nogueira.br/,"Estrada Luiz Otávio Marques, 69",71414-689,Fazenda de da Rocha,15,Alto do Moura,Caruaru,PE,Brasil,2024-03-20
3,132980be-6003-45cf-a455-76149b2bebe7,Bruno Rezende,740.619.382-31,1961-12-19,Outro,Outro,Amarela,Ensino Fundamental,Divorciado,1 a 3 salários mínimos,...,http://casa.br/,"Ladeira de Cavalcante, 60",36704-407,Avenida de Silva,482,Cedro,Caruaru,PE,Brasil,2024-03-16
4,66916408-9b4b-42cf-9e84-c87ad0e398ab,Isabela da Cunha,207.493.865-92,1941-07-25,Feminino,Outro,Preta,Ensino Médio,Viúvo,1 a 3 salários mínimos,...,http://www.da.com/,"Praça Pinto, 89",45229299,Lago de Dias,45,Petrópolis,Caruaru,PE,Brasil,2024-04-29


**Ou assim:**

In [ ]:
query = """
SELECT *
FROM CAD_PESSOA
"""
# Utilizando o método read_sql_query do pandas:
df_cad_pessoa = pd.read_sql_query(query, conn)
df_cad_pessoa.head()

,ID_PESSOA,NOME,CPF,DATA_NASC,GENERO,ORIENTACAO_SEXUAL,RACA_ETNIA,ESCOLARIDADE,ESTADO_CIVIL,FAIXA_DE_RENDA,...,URL_REDE_SOCIAL,ENDERECO,CEP,LOGRADOURO,NUMERO,BAIRRO,CIDADE,ESTADO,PAIS,DATA_VISITA
0,713970c2-540c-484d-b4c5-e432e5cce09c,Juan Teixeira,601.873.924-04,1958-06-20,Masculino,Heterossexual,Parda,Ensino Fundamental,Solteiro,Mais de 5 salários mínimos,...,https://rios.net/,"Núcleo da Paz, 96",27656106,Jardim Caldeira,139,Santa Clara,Caruaru,PE,Brasil,2024-03-19
1,25d88136-001f-4e37-8e2a-3de3f56bbf2c,Francisco Aragão,845.706.913-66,1950-07-25,Feminino,Bissexual,Preta,Ensino Superior,Casado,3 a 5 salários mínimos,...,http://vieira.br/,"Praia Gabriela Pastor, 89",46065-772,Residencial João Sales,70,São José,Caruaru,PE,Brasil,2024-03-10
2,83482d24-7cf1-4335-9f3a-c267e6b9f2c8,Otto Peixoto,867.051.243-26,1972-08-20,Outro,Outro,Preta,Ensino Fundamental,Casado,1 a 3 salários mínimos,...,http://nogueira.br/,"Estrada Luiz Otávio Marques, 69",71414-689,Fazenda de da Rocha,15,Alto do Moura,Caruaru,PE,Brasil,2024-03-20
3,132980be-6003-45cf-a455-76149b2bebe7,Bruno Rezende,740.619.382-31,1961-12-19,Outro,Outro,Amarela,Ensino Fundamental,Divorciado,1 a 3 salários mínimos,...,http://casa.br/,"Ladeira de Cavalcante, 60",36704-407,Avenida de Silva,482,Cedro,Caruaru,PE,Brasil,2024-03-16
4,66916408-9b4b-42cf-9e84-c87ad0e398ab,Isabela da Cunha,207.493.865-92,1941-07-25,Feminino,Outro,Preta,Ensino Médio,Viúvo,1 a 3 salários mínimos,...,http://www.da.com/,"Praça Pinto, 89",45229299,Lago de Dias,45,Petrópolis,Caruaru,PE,Brasil,2024-04-29


In [ ]:
conn.close()

## Consultando o Banco utilizando SQL diretamente

In [ ]:
%load_ext sql
%sql sqlite:///{db_path}

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql
SELECT *
FROM CAD_PESSOA
LIMIT 5

 * sqlite:////content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db
Done.


ID_PESSOA,NOME,CPF,DATA_NASC,GENERO,ORIENTACAO_SEXUAL,RACA_ETNIA,ESCOLARIDADE,ESTADO_CIVIL,FAIXA_DE_RENDA,AREA_DE_ATUACAO,TELEFONE,EMAIL,URL_REDE_SOCIAL,ENDERECO,CEP,LOGRADOURO,NUMERO,BAIRRO,CIDADE,ESTADO,PAIS,DATA_VISITA
713970c2-540c-484d-b4c5-e432e5cce09c,Juan Teixeira,601.873.924-04,1958-06-20,Masculino,Heterossexual,Parda,Ensino Fundamental,Solteiro,Mais de 5 salários mínimos,Tecnologia,84 5226 9451,rbarbosa@example.org,https://rios.net/,"Núcleo da Paz, 96",27656106,Jardim Caldeira,139,Santa Clara,Caruaru,PE,Brasil,2024-03-19
25d88136-001f-4e37-8e2a-3de3f56bbf2c,Francisco Aragão,845.706.913-66,1950-07-25,Feminino,Bissexual,Preta,Ensino Superior,Casado,3 a 5 salários mínimos,Tecnologia,0800 798 0125,arthur-gabriel33@example.com,http://vieira.br/,"Praia Gabriela Pastor, 89",46065-772,Residencial João Sales,70,São José,Caruaru,PE,Brasil,2024-03-10
83482d24-7cf1-4335-9f3a-c267e6b9f2c8,Otto Peixoto,867.051.243-26,1972-08-20,Outro,Outro,Preta,Ensino Fundamental,Casado,1 a 3 salários mínimos,Serviços,(021) 0842-3021,ayla19@example.net,http://nogueira.br/,"Estrada Luiz Otávio Marques, 69",71414-689,Fazenda de da Rocha,15,Alto do Moura,Caruaru,PE,Brasil,2024-03-20
132980be-6003-45cf-a455-76149b2bebe7,Bruno Rezende,740.619.382-31,1961-12-19,Outro,Outro,Amarela,Ensino Fundamental,Divorciado,1 a 3 salários mínimos,Serviços,+55 71 4127-8733,wsousa@example.com,http://casa.br/,"Ladeira de Cavalcante, 60",36704-407,Avenida de Silva,482,Cedro,Caruaru,PE,Brasil,2024-03-16
66916408-9b4b-42cf-9e84-c87ad0e398ab,Isabela da Cunha,207.493.865-92,1941-07-25,Feminino,Outro,Preta,Ensino Médio,Viúvo,1 a 3 salários mínimos,Educação,+55 84 7698 8134,esthervargas@example.org,http://www.da.com/,"Praça Pinto, 89",45229299,Lago de Dias,45,Petrópolis,Caruaru,PE,Brasil,2024-04-29


## Misturando o modo SQL com o modo Python

In [ ]:
cad_pessoa_df = %sql SELECT * FROM CAD_PESSOA
cad_pessoa_df = cad_pessoa_df.DataFrame()
cad_pessoa_df.head()

 * sqlite:////content/drive/MyDrive/colab-notebooks/Projetos/oa-ac-sql-intro/data/ac_fake_db.db
Done.


,ID_PESSOA,NOME,CPF,DATA_NASC,GENERO,ORIENTACAO_SEXUAL,RACA_ETNIA,ESCOLARIDADE,ESTADO_CIVIL,FAIXA_DE_RENDA,...,URL_REDE_SOCIAL,ENDERECO,CEP,LOGRADOURO,NUMERO,BAIRRO,CIDADE,ESTADO,PAIS,DATA_VISITA
0,713970c2-540c-484d-b4c5-e432e5cce09c,Juan Teixeira,601.873.924-04,1958-06-20,Masculino,Heterossexual,Parda,Ensino Fundamental,Solteiro,Mais de 5 salários mínimos,...,https://rios.net/,"Núcleo da Paz, 96",27656106,Jardim Caldeira,139,Santa Clara,Caruaru,PE,Brasil,2024-03-19
1,25d88136-001f-4e37-8e2a-3de3f56bbf2c,Francisco Aragão,845.706.913-66,1950-07-25,Feminino,Bissexual,Preta,Ensino Superior,Casado,3 a 5 salários mínimos,...,http://vieira.br/,"Praia Gabriela Pastor, 89",46065-772,Residencial João Sales,70,São José,Caruaru,PE,Brasil,2024-03-10
2,83482d24-7cf1-4335-9f3a-c267e6b9f2c8,Otto Peixoto,867.051.243-26,1972-08-20,Outro,Outro,Preta,Ensino Fundamental,Casado,1 a 3 salários mínimos,...,http://nogueira.br/,"Estrada Luiz Otávio Marques, 69",71414-689,Fazenda de da Rocha,15,Alto do Moura,Caruaru,PE,Brasil,2024-03-20
3,132980be-6003-45cf-a455-76149b2bebe7,Bruno Rezende,740.619.382-31,1961-12-19,Outro,Outro,Amarela,Ensino Fundamental,Divorciado,1 a 3 salários mínimos,...,http://casa.br/,"Ladeira de Cavalcante, 60",36704-407,Avenida de Silva,482,Cedro,Caruaru,PE,Brasil,2024-03-16
4,66916408-9b4b-42cf-9e84-c87ad0e398ab,Isabela da Cunha,207.493.865-92,1941-07-25,Feminino,Outro,Preta,Ensino Médio,Viúvo,1 a 3 salários mínimos,...,http://www.da.com/,"Praça Pinto, 89",45229299,Lago de Dias,45,Petrópolis,Caruaru,PE,Brasil,2024-04-29
